# Final Predictions

Now that we have a final model we're going to predict 2020 games for this coming week! 

In [1]:
#import functions 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from yellowbrick.classifier import ConfusionMatrix
import seaborn as sns
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000
import sqlite3
from keras import layers 
from keras import models 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential 
from sklearn.preprocessing import StandardScaler
from keras import optimizers

Using TensorFlow backend.


In [2]:
#upload dataset
conn = sqlite3.connect('NFL_Modeling_Stats.db')
c = conn.cursor()

In [3]:
#preview dataset
query = """
SELECT * 
FROM modeled_stats
"""

#make dataframe
data = pd.read_sql_query(query, conn)

#preview
data.head()

,index,Home_Team,Away_Team,Home_Week,Away_Week,Winner,Home_Pts_Scored,Away_Pts_Scored,Home_Pts_Al,Away_Pts_Al,Home_Win,Away_Win,Home_TDs,Away_TDs,Home_First_Sc,Away_First_Sc,Home_Second_Sc,Away_Second_Sc,Home_Third_Sc,Away_Third_Sc,Home_Fourth_Sc,Away_Fourth_Sc,Home_First_Al,Away_First_Al,Home_Second_Al,Away_Second_Al,Home_Third_Al,Away_Third_Al,Home_Fourth_Al,Away_Fourth_Al,Home_Off_Plys,Away_Off_Plys,Home_Rush_Plys,Away_Rush_Plys,Home_Pass_Plys,Away_Pass_Plys,Home_Fumbles,Away_Fumbles,Home_Yds_Ply,Away_Yds_Ply,Home_Yds_Rush,Away_Yds_Rush,Home_Yds_Pass,Away_Yds_Pass,Home_Total_Yds_Gm,Away_Total_Yds_Gm,Home_Total_Rush_Yds_Gm,Away_Total_Rush_Yds_Gm,Home_Total_Pass_Yds_Gm,Away_Total_Pass_Yds_Gm,Home_Yds_Ply_Al,Away_Yds_Ply_Al,Home_Yds_Rush_Al,Away_Yds_Rush_Al,Home_Yds_Pass_Al,Away_Yds_Pass_Al,Home_Total_Yds_Gm_Al,Away_Total_Yds_Gm_Al,Home_Total_Rush_Yds_Gm_Al,Away_Total_Rush_Yds_Gm_Al,Home_Total_Pass_Yds_Gm_Al,Away_Total_Pass_Yds_Gm_Al,Home_Off_Plys_Al,Away_Off_Plys_Al,Home_Rush_Plys_Al,Away_Rush_Plys_Al,Home_Pass_Plys_Al,Away_Pass_Plys_Al,Home_Completion_Rate,Away_Completion_Rate,Home_Pass_Att,Away_Pass_Att,Home_Pass_TDs,Away_Pass_TDs,Home_INTs,Away_INTs,Home_Sacks_Taken,Away_Sacks_Taken,Home_Rush_TDs,Away_Rush_TDs,Home_Rush_Att,Away_Rush_Att,Home_Pass_1Ds_Gm,Away_Pass_1Ds_Gm,Home_Rush_1Ds_Gm,Away_Rush_1Ds_Gm,Home_First_Ds_Gm,Away_First_Ds_Gm,Home_Third_Conv_Perc,Away_Third_Conv_Perc,Home_Fourth_Conv_Perc,Away_Fourth_Conv_Perc,Home_Sacks_Gm,Away_Sacks_Gm,Home_INT_Gm,Away_INT_Gm,Home_Fumbles_Caused,Away_Fumbles_Caused,Home_TO_Gm,Away_TO_Gm,Home_FG_Att,Away_FG_Att,Home_FG_Perc_Comp,Away_FG_Perc_Comp,Home_PAT_Att,Away_PAT_Att,Home_PAT_Perc_Comp,Away_PAT_Perc_Comp,Home_Punts,Away_Punts,Home_Avg_Punt_Yds_Gm,Away_Avg_Punt_Yds_Gm,Home_Avg_Punt_Ret_Yds_Gm,Away_Avg_Punt_Ret_Yds_Gm,Home_Avg_Punt_Ret_Yds_Opp_Gm,Away_Avg_Punt_Ret_Yds_Opp_Gm,Home_Total_Punt_Ret_Yds_Gm,Away_Total_Punt_Ret_Yds_Gm,Home_Total_Punt_Ret_Yds_Opp_Gm,Away_Total_Punt_Ret_Yds_Opp_Gm,Home_Punt_Ret_TD,Away_Punt_Ret_TD,Home_Punt_Ret_TD_Opp,Away_Punt_Ret_TD_Opp,Home_Avg_Kick_Ret_Yds_Gm,Away_Avg_Kick_Ret_Yds_Gm,Home_Avg_Kick_Ret_Yds_Opp_Gm,Away_Avg_Kick_Ret_Yds_Opp_Gm,Home_Kick_Ret_TD_Opp,Away_Kick_Ret_TD_Opp,Home_Kick_Ret_TD,Away_Kick_Ret_TD,Home_Penalty_Gm,Away_Penalty_Gm,Home_Penalty_Yds_Gm,Away_Penalty_Yds_Gm,Home_Playoff_app_5_yrs,Away_Playoff_app_5_yrs,Home_Draft_Pos,Away_Draft_Pos,Home_Week_Of_Szn,Away_Week_Of_Szn,Home_Perc_Pro_Bowl,Away_Perc_Pro_Bowl,Home_Year,Away_Year
0,Week_1_Bears,Bears,Packers,1.0,1.0,Away,3.0,10.0,10.0,3.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,7.0,0.0,0.0,0.0,3.0,0.0,3.0,7.0,0.0,0.0,0.0,3.0,0.0,65.0,57.0,15.0,22.0,45.0,30.0,0.0,0.0,3.9,3.7,3.1,2.1,5.1,6.8,254.0,213.0,46.0,47.0,208.0,166.0,3.9,3.7,2.1,3.1,6.8,5.1,213.0,254.0,47.0,46.0,166.0,208.0,57.0,65.0,22.0,15.0,30.0,45.0,0.577778,0.600000,45.0,30.0,0.0,1.0,1.0,0.0,5.0,5.0,0.0,0.0,15.0,22.0,10.0,10.0,3.0,1.0,16.0,13.0,20.0,16.0,0.0,0.0,5.0,5.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,8.0,9.0,42.6,47.6,9.0,1.0,1.0,9.0,36.0,1.0,1.0,36.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,10.0,10.0,107.0,71.0,1.0,7.0,73.0,30.0,1.0,1.0,0.094340,0.056604,2019.0,2019.0
1,Week_1_Eagles,Eagles,Redskins,1.0,1.0,Home,32.0,27.0,27.0,32.0,1.0,0.0,4.0,3.0,0.0,10.0,7.0,10.0,14.0,0.0,11.0,7.0,10.0,0.0,10.0,7.0,0.0,14.0,7.0,11.0,71.0,58.0,31.0,13.0,39.0,44.0,0.0,0.0,6.1,6.9,4.0,2.2,8.0,8.6,436.0,398.0,123.0,28.0,313.0,370.0,6.1,6.9,2.2,4.0,8.6,8.0,398.0,436.0,28.0,123.0,370.0,313.0,58.0,71.0,13.0,31.0,44.0,39.0,0.717949,0.681818,39.0,44.0,3.0,3.0,0.0,0.0,1.0,1.0,1.0,0.0,31.0,13.0,13.0,15.0,9.0,0.0,22.0,15.0,64.0,38.0,50.0,100.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,5.0,51.3,54.4,11.5,11.0,11.0,11.5,46.0,11.0,11.0,46.0,0.0,0.0,0.0,0.0,0.000000,15.333333,15.333333,0.000000,0.0,0.0,0.0,0.0,6.0,12.0,55.0,96.0,5.0,0.0,22.0,15.0,1.0,1.0,0.113208,0.037736,2019.0,2019.0
2,Week_1_Jets,Jets,Bills,1.0,1.0,Away,16.0,17.0,17.0,16.0,0.0,1.0,1.0,2.0,6.0,0.0,0.0,0.0,10.0

In [4]:
data = data.drop('index', axis=1)

In [5]:
#uploading raw data
conn_raw = sqlite3.connect('NFL_Stats.db')
c_2 = conn_raw.cursor()

query_2 = """
SELECT * 
FROM stats_2019
"""
query_3 = """
SELECT * 
FROM stats_2020
"""
#make dataframe
year_1 = pd.read_sql_query(query_2, conn_raw)
year_2 = pd.read_sql_query(query_2, conn_raw)

raw_data = pd.concat([year_1,year_2])
raw_data = raw_data.drop(['index','Penalties', 'Opp_Name', 'Superbowl_Win','Year'], axis=1)

In [6]:
#upload model
from keras.models import model_from_json

json_file = open('final_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
nfl_model = model_from_json(loaded_model_json)

#add weights
nfl_model.load_weights("final_model.h5")

In [7]:
def Get_Table(Week_Dict, Week):
    columns = raw_data.columns.drop('Team_Name')
    df = pd.DataFrame()
    for x,y in enumerate(Week_Dict.items()):
        Week = Week
        Game = 'Week_'+str(Week)+'_'+y[1]
        df.loc[Game, 'Home_Team'] = y[1]
        df.loc[Game, 'Away_Team'] = y[0]
        if raw_data.loc[(raw_data['Team_Name'] == y[1]) & (raw_data['Week_Of_Szn'] == Week)]['Win'].sum() == 0.0:
            df.loc[Game, 'Winner'] = 'Away'
        else:
            df.loc[Game, 'Winner'] = 'Home'
        for column in columns:
            df.loc[Game, 'Home_'+column] = raw_data.loc[(raw_data['Team_Name'] == y[1]) &
                                                   (raw_data['Week_Of_Szn'] <= Week)][column].mean()
            df.loc[Game, 'Away_'+column] = raw_data.loc[(raw_data['Team_Name'] == y[0]) & 
                                                              (raw_data['Week_Of_Szn'] <= Week)][column].mean()

    
    return df

Now I'd like to create a function in which you can input the two team names, week, and year you're interested in and recieve a prediction! 

In [8]:
def Predict(Home_Team, Away_Team, Year, Week):
    #features from boruta 
    feats = ['Away_Pts_Scored',
     'Home_Pts_Al',
     'Away_Pts_Al',
     'Home_TDs',
     'Home_Rush_Plys_Al',
     'Away_Rush_Plys_Al',
     'Away_PAT_Att',
     'Away_Yds_Pass_Al',
     'Home_Pass_Plys_Al']

    #preprocessing
    Dummy = pd.get_dummies(data, columns=['Winner'])
    
    first = Dummy.loc[(Dummy['Home_Year'] == Year) & (Dummy['Home_Week'] < Week)]
    second = Dummy.loc[(Dummy['Home_Year'] < Year)]
    final = pd.concat([first, second])
    
    #deal with missing data 
    final = final.fillna(final.mean())
    
    #train test split
    X = final[feats]
    y = final.Winner_Home

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11, test_size=.2) 
    
    #standard scaling

    scaler = StandardScaler().fit(X_train)

    X_train = scaler.transform(X_train)

    X_test = scaler.transform(X_test)

    #model compilation

    nfl_model.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

    #fit
    nfl_model.fit(X_train, y_train,epochs=50, batch_size=16, verbose=1, validation_data=(X_test, y_test))

    #evaluate
    y_pred = nfl_model.predict(X_test)
    score = nfl_model.evaluate(X_test, y_test,verbose=1)
    print('\n''The accuracy of the current model is:',score[1])

    ####predict 
    Week_Pred = {Away_Team : Home_Team}
    pred_line = Get_Table(Week_Pred, Week)
    pred_dummies = pd.get_dummies(pred_line, columns=['Winner'])
    pred_final = pred_dummies[feats]

    final_pred = nfl_model.predict_proba(pred_final)
    
    return print("The chance that the", Home_Team, "win is:", (final_pred[0][0]*100),'%')

In [9]:
Predict('Redskins','Giants', 2020, 9)

Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 0s 753us/step - loss: 0.5586 - acc: 0.7500 - val_loss: 0.5248 - val_acc: 0.7143
Epoch 2/50
308/308 [==============================] - 0s 125us/step - loss: 0.5269 - acc: 0.7597 - val_loss: 0.5264 - val_acc: 0.7013
Epoch 3/50
308/308 [==============================] - 0s 125us/step - loss: 0.5138 - acc: 0.7760 - val_loss: 0.5284 - val_acc: 0.7013
Epoch 4/50
308/308 [==============================] - 0s 132us/step - loss: 0.5475 - acc: 0.7435 - val_loss: 0.5237 - val_acc: 0.7013
Epoch 5/50
308/308 [==============================] - 0s 124us/step - loss: 0.5005 - acc: 0.7825 - val_loss: 0.5176 - val_acc: 0.6883
Epoch 6/50
308/308 [==============================] - 0s 133us/step - loss: 0.5304 - acc: 0.7597 - val_loss: 0.5137 - val_acc: 0.6753
Epoch 7/50
308/308 [==============================] - 0s 114us/step - loss: 0.5262 - acc: 0.7532 - val_loss: 0.5173 - val_acc: 0.6883
Epoch 8/50
308/30

In [10]:
def Predict_Win(Home_Team, Away_Team, Year, Week):
    
    #features from boruta 
    #First I've got to get dummy values for 
    NFL_Dummies = pd.get_dummies(data, columns=['Winner'])
    NFL_Dummies = NFL_Dummies.fillna(NFL_Dummies.mean())

    X = NFL_Dummies.drop(['Winner_Home','Winner_Away','Home_Team','Away_Team','Home_Win','Away_Win'], axis=1)
    y = NFL_Dummies.Winner_Home

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11, test_size=.2)
    
    #boruta
    from boruta import BorutaPy

    ###initialize Boruta
    forest = RandomForestRegressor(
       n_jobs = -1, 
       max_depth = 5
    )

    boruta = BorutaPy(
       estimator = forest, 
       n_estimators = 50,
       max_iter = 200 # number of trials to perform
    )

    ### fit Boruta (it accepts np.array, not pd.DataFrame)
    boruta.fit(np.array(X), np.array(y))

    ### print results

    green_area = X.columns[boruta.support_].to_list()
    blue_area = X.columns[boruta.support_weak_].to_list()
    feats = green_area + blue_area
    print('The features used in this model were:', feats)

    #preprocessing
    Dummy = pd.get_dummies(data, columns=['Winner'])
    
    first = Dummy.loc[(Dummy['Home_Year'] == Year) & (Dummy['Home_Week'] < Week)]
    second = Dummy.loc[(Dummy['Home_Year'] < Year)]
    final = pd.concat([first, second])
    
    #deal with missing data 
    final = final.fillna(final.mean())
    
    #train test split
    X = final[feats]
    y = final.Winner_Home

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11, test_size=.2) 
    
    np.random.seed(8)

    # CNN model 

    model_1 = Sequential()

    model_1.add(Dropout(.2, input_shape=(len(feats),)))

    model_1.add(Dense(64, activation='relu'))

    model_1.add(Dense(56, activation='relu'))

    model_1.add(Dense(40, activation='relu'))

    model_1.add(Dense(32, activation='relu'))

    model_1.add(Dropout(.3))

    model_1.add(Dense(8, activation='relu'))

    model_1.add(Dense(1, activation='sigmoid'))
    
    #standard scaling

    scaler = StandardScaler().fit(X_train)

    X_train = scaler.transform(X_train)

    X_test = scaler.transform(X_test)

    #model compilation

    model_1.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

    #fit
    model_1.fit(X_train, y_train,epochs=50, batch_size=16, verbose=1, validation_data=(X_test, y_test))

    #evaluate
    y_pred = model_1.predict(X_test)
    score = model_1.evaluate(X_test, y_test,verbose=1)
    print('\n''The accuracy of the current model is:',score[1])

    ####predict 
    Week_Pred = {Away_Team : Home_Team}
    pred_line = Get_Table(Week_Pred, Week)
    pred_dummies = pd.get_dummies(pred_line, columns=['Winner'])
    pred_final = pred_dummies[feats]

    final_pred = model_1.predict_proba(pred_final)
    
    return print("The chance that the", Home_Team, "win is:", (final_pred[0][0]*100),'%')

In [11]:
Predict_Win('Redskins','Giants',2020,9)

The features used in this model were: ['Home_Pts_Scored', 'Away_Pts_Scored', 'Home_Pts_Al', 'Away_Pts_Al', 'Home_TDs', 'Away_Second_Al', 'Home_Rush_Plys_Al', 'Away_Rush_Plys_Al', 'Home_Pass_Att', 'Away_INTs', 'Home_Sacks_Gm', 'Home_PAT_Att', 'Away_PAT_Att', 'Away_Pass_Plys_Al']
Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 0s 978us/step - loss: 0.6773 - acc: 0.4870 - val_loss: 0.6633 - val_acc: 0.6234
Epoch 2/50
308/308 [==============================] - 0s 125us/step - loss: 0.6682 - acc: 0.5714 - val_loss: 0.6601 - val_acc: 0.6364
Epoch 3/50
308/308 [==============================] - 0s 138us/step - loss: 0.6521 - acc: 0.5909 - val_loss: 0.6524 - val_acc: 0.6234
Epoch 4/50
308/308 [==============================] - 0s 131us/step - loss: 0.6488 - acc: 0.6201 - val_loss: 0.6477 - val_acc: 0.7013
Epoch 5/50
308/308 [==============================] - 0s 143us/step - loss: 0.6547 - acc: 0.5974 - val_loss: 0.6391 - val_acc: 0.7013
Epoch 

In [12]:
Predict_Win('Cowboys','Steelers', 2020, 9)

The features used in this model were: ['Home_Pts_Scored', 'Away_Pts_Scored', 'Home_Pts_Al', 'Away_Pts_Al', 'Away_Second_Al', 'Home_Rush_Plys_Al', 'Away_Rush_Plys_Al', 'Away_Pass_Plys_Al', 'Away_INTs', 'Home_Sacks_Gm', 'Home_PAT_Att', 'Away_PAT_Att', 'Home_TDs']
Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 0s 1ms/step - loss: 0.7251 - acc: 0.4708 - val_loss: 0.6941 - val_acc: 0.5195
Epoch 2/50
308/308 [==============================] - 0s 105us/step - loss: 0.7032 - acc: 0.5422 - val_loss: 0.6832 - val_acc: 0.6234
Epoch 3/50
308/308 [==============================] - 0s 104us/step - loss: 0.6841 - acc: 0.5617 - val_loss: 0.6742 - val_acc: 0.6494
Epoch 4/50
308/308 [==============================] - 0s 94us/step - loss: 0.6694 - acc: 0.6266 - val_loss: 0.6658 - val_acc: 0.7143
Epoch 5/50
308/308 [==============================] - 0s 91us/step - loss: 0.6659 - acc: 0.6266 - val_loss: 0.6557 - val_acc: 0.7143
Epoch 6/50
308/308 [=======

In [13]:
Predict_Win('49ers','Packers', 2020, 9)

The features used in this model were: ['Home_Pts_Scored', 'Away_Pts_Scored', 'Home_Pts_Al', 'Away_Pts_Al', 'Away_Second_Al', 'Home_Rush_Plys_Al', 'Away_Rush_Plys_Al', 'Away_Pass_Plys_Al', 'Away_INTs', 'Home_Sacks_Gm', 'Home_PAT_Att', 'Away_PAT_Att', 'Home_TDs']
Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 0s 2ms/step - loss: 0.7251 - acc: 0.4708 - val_loss: 0.6941 - val_acc: 0.5195
Epoch 2/50
308/308 [==============================] - 0s 216us/step - loss: 0.7032 - acc: 0.5422 - val_loss: 0.6832 - val_acc: 0.6234
Epoch 3/50
308/308 [==============================] - 0s 167us/step - loss: 0.6842 - acc: 0.5617 - val_loss: 0.6744 - val_acc: 0.6494
Epoch 4/50
308/308 [==============================] - 0s 125us/step - loss: 0.6695 - acc: 0.6266 - val_loss: 0.6659 - val_acc: 0.7143
Epoch 5/50
308/308 [==============================] - 0s 133us/step - loss: 0.6660 - acc: 0.6299 - val_loss: 0.6558 - val_acc: 0.7143
Epoch 6/50
308/308 [=====

In [14]:
Predict('49ers','Packers', 2020, 9)

Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 0s 1ms/step - loss: 0.4649 - acc: 0.7662 - val_loss: 0.5335 - val_acc: 0.7143
Epoch 2/50
308/308 [==============================] - 0s 141us/step - loss: 0.4719 - acc: 0.7792 - val_loss: 0.5290 - val_acc: 0.7273
Epoch 3/50
308/308 [==============================] - 0s 130us/step - loss: 0.4645 - acc: 0.8052 - val_loss: 0.5312 - val_acc: 0.7273
Epoch 4/50
308/308 [==============================] - 0s 152us/step - loss: 0.4585 - acc: 0.7695 - val_loss: 0.5267 - val_acc: 0.7273
Epoch 5/50
308/308 [==============================] - 0s 131us/step - loss: 0.4455 - acc: 0.7662 - val_loss: 0.5327 - val_acc: 0.7143
Epoch 6/50
308/308 [==============================] - 0s 127us/step - loss: 0.4422 - acc: 0.8052 - val_loss: 0.5221 - val_acc: 0.7273
Epoch 7/50
308/308 [==============================] - 0s 138us/step - loss: 0.4539 - acc: 0.7792 - val_loss: 0.5225 - val_acc: 0.7273
Epoch 8/50
308/308 

In [15]:
Predict('Cowboys','Steelers', 2020, 9)

Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 1s 2ms/step - loss: 0.4251 - acc: 0.7987 - val_loss: 0.5460 - val_acc: 0.7013
Epoch 2/50
308/308 [==============================] - 0s 218us/step - loss: 0.4448 - acc: 0.7792 - val_loss: 0.5361 - val_acc: 0.7013
Epoch 3/50
308/308 [==============================] - 0s 212us/step - loss: 0.4078 - acc: 0.8247 - val_loss: 0.5409 - val_acc: 0.7013
Epoch 4/50
308/308 [==============================] - 0s 216us/step - loss: 0.4438 - acc: 0.7922 - val_loss: 0.5484 - val_acc: 0.7013
Epoch 5/50
308/308 [==============================] - 0s 228us/step - loss: 0.4254 - acc: 0.8117 - val_loss: 0.5323 - val_acc: 0.7013
Epoch 6/50
308/308 [==============================] - 0s 216us/step - loss: 0.4404 - acc: 0.8149 - val_loss: 0.5321 - val_acc: 0.7013
Epoch 7/50
308/308 [==============================] - 0s 205us/step - loss: 0.4482 - acc: 0.8084 - val_loss: 0.5304 - val_acc: 0.7013
Epoch 8/50
308/308 

In [16]:
Predict_Win('Chiefs','Panthers', 2020, 9)

The features used in this model were: ['Home_Pts_Scored', 'Away_Pts_Scored', 'Home_Pts_Al', 'Away_Pts_Al', 'Home_TDs', 'Away_Second_Al', 'Home_Rush_Plys_Al', 'Away_Rush_Plys_Al', 'Away_INTs', 'Home_Sacks_Gm', 'Home_PAT_Att', 'Away_PAT_Att', 'Away_Pass_Plys_Al', 'Home_Pass_Att']
Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 1s 2ms/step - loss: 0.6755 - acc: 0.5162 - val_loss: 0.6554 - val_acc: 0.6494
Epoch 2/50
308/308 [==============================] - 0s 250us/step - loss: 0.6659 - acc: 0.5844 - val_loss: 0.6524 - val_acc: 0.6364
Epoch 3/50
308/308 [==============================] - 0s 267us/step - loss: 0.6467 - acc: 0.6299 - val_loss: 0.6423 - val_acc: 0.6883
Epoch 4/50
308/308 [==============================] - 0s 273us/step - loss: 0.6422 - acc: 0.6656 - val_loss: 0.6364 - val_acc: 0.6623
Epoch 5/50
308/308 [==============================] - 0s 324us/step - loss: 0.6414 - acc: 0.6526 - val_loss: 0.6263 - val_acc: 0.6753
Epoch 6/

In [17]:
Predict('Chiefs','Panthers', 2020, 9)

Train on 308 samples, validate on 77 samples
Epoch 1/50
308/308 [==============================] - 1s 2ms/step - loss: 0.4128 - acc: 0.7922 - val_loss: 0.5513 - val_acc: 0.6753
Epoch 2/50
308/308 [==============================] - 0s 235us/step - loss: 0.3879 - acc: 0.8149 - val_loss: 0.5550 - val_acc: 0.6753
Epoch 3/50
308/308 [==============================] - 0s 296us/step - loss: 0.4154 - acc: 0.7955 - val_loss: 0.5573 - val_acc: 0.6753
Epoch 4/50
308/308 [==============================] - 0s 305us/step - loss: 0.3916 - acc: 0.8247 - val_loss: 0.5555 - val_acc: 0.6883
Epoch 5/50
308/308 [==============================] - 0s 358us/step - loss: 0.4133 - acc: 0.8247 - val_loss: 0.5499 - val_acc: 0.6883
Epoch 6/50
308/308 [==============================] - 0s 314us/step - loss: 0.4275 - acc: 0.7890 - val_loss: 0.5459 - val_acc: 0.6494
Epoch 7/50
308/308 [==============================] - 0s 319us/step - loss: 0.4196 - acc: 0.8084 - val_loss: 0.5511 - val_acc: 0.6494
Epoch 8/50
308/308 

In [18]:
Predict_Win('Chiefs','Texans', 2020,1)

The features used in this model were: ['Home_Pts_Scored', 'Away_Pts_Scored', 'Home_Pts_Al', 'Away_Pts_Al', 'Home_TDs', 'Away_Second_Al', 'Home_Rush_Plys_Al', 'Away_Rush_Plys_Al', 'Away_Pass_Plys_Al', 'Away_INTs', 'Home_Sacks_Gm', 'Home_PAT_Att', 'Away_PAT_Att']
Train on 213 samples, validate on 54 samples
Epoch 1/50
213/213 [==============================] - 1s 3ms/step - loss: 0.6661 - acc: 0.5728 - val_loss: 0.6632 - val_acc: 0.5741
Epoch 2/50
213/213 [==============================] - 0s 256us/step - loss: 0.6888 - acc: 0.5117 - val_loss: 0.6604 - val_acc: 0.5926
Epoch 3/50
213/213 [==============================] - 0s 261us/step - loss: 0.6662 - acc: 0.5493 - val_loss: 0.6567 - val_acc: 0.6296
Epoch 4/50
213/213 [==============================] - 0s 249us/step - loss: 0.6649 - acc: 0.5775 - val_loss: 0.6532 - val_acc: 0.6296
Epoch 5/50
213/213 [==============================] - 0s 377us/step - loss: 0.6640 - acc: 0.6150 - val_loss: 0.6487 - val_acc: 0.6111
Epoch 6/50
213/213 [=====

['Home_Pts_Scored', 'Away_Pts_Scored', 'Home_Pts_Al', 'Away_Pts_Al', 'Home_TDs', 'Away_Second_Al', 'Home_Rush_Plys_Al', 'Away_Rush_Plys_Al', 'Away_INTs', 'Home_Sacks_Gm', 'Home_PAT_Att', 'Away_PAT_Att', 'Away_Pass_Plys_Al', 'Home_Pass_Att']